In [1]:

import glob

import numpy as np
import pandas as pd
import os
import json
import ast
import tqdm
import concurrent.futures

In [12]:
# Openpose features
def get_openpose_from_json(video_id, start_frame, window_size):
    """
    :param video_id: the id of video you want to get openpose
    :param start_frame: the frame when the bite action starts
    :param window_size: the window size of each sample in seconds
    """

    if os.path.exists(f"/home/aa2375/social-dining/data/openpose/processed/{video_id}_{start_frame}.npy"):
        d = np.load(f"/home/aa2375/social-dining/data/openpose/processed/{video_id}_{start_frame}.npy", allow_pickle=True).item()
        body_sample = d['body']
        face_sample = d['face']
        
    else:
        path = '/home/aa2375/social-dining/data/openpose/' + str(video_id)
        # frame_list = os.listdir(path)
        frame_list = glob.glob(path + '/*')
        frame_list.sort()
        # print("len:", len(frame_list))

        b_x, b_y, b_c = [], [], []
        lh_x, lh_y, lh_c = [], [], []
        rh_x, rh_y, rh_c = [], [], []
        f_x, f_y, f_c = [], [], []

        start_id = start_frame
        end_id = start_frame + 30 * window_size

        for i in range(start_frame, end_id):
            with open(frame_list[i]) as f:
                pose_frame = json.load(f)
            if not pose_frame['people']:  # if no body detected:
                body_joints = np.empty(25 * 3)
                body_joints[:] = np.nan
                face_joints = np.empty(70 * 3)
                face_joints[:] = np.nan
            else:
                body_joints = pose_frame['people'][0]['pose_keypoints_2d']
                face_joints = pose_frame['people'][0]['face_keypoints_2d']

            a = 0
            while a < (len(body_joints)):
                b_x.append(body_joints[a])
                b_y.append(body_joints[a + 1])
                b_c.append(body_joints[a + 1])
                a += 3
            a = 0
            while a < (len(face_joints)):
                f_x.append(face_joints[a])
                f_y.append(face_joints[a + 1])
                f_c.append(face_joints[a + 1])
                a += 3

        body_op_x = np.array(b_x).reshape(window_size * 30, -1)
        body_op_y = np.array(b_y).reshape(window_size * 30, -1)
        body_op_c = np.array(b_c).reshape(window_size * 30, -1)

        # remove keypoints on legs and feet
        body_op_x = np.delete(body_op_x, [10, 11, 13, 14, 18, 19, 20, 21, 22, 23, 24], 1)
        body_op_y = np.delete(body_op_y, [10, 11, 13, 14, 18, 19, 20, 21, 22, 23, 24], 1)
        body_op_c = np.delete(body_op_c, [10, 11, 13, 14, 18, 19, 20, 21, 22, 23, 24], 1)


        face_op_x = np.array(f_x).reshape(window_size * 30, -1)
        face_op_y = np.array(f_y).reshape(window_size * 30, -1)
        face_op_c = np.array(f_c).reshape(window_size * 30, -1)

        body_sample = np.concatenate([body_op_x, body_op_y], axis=1)
        face_sample = np.concatenate([face_op_x, face_op_y], axis=1)

        np.save(f'/home/aa2375/social-dining/data/openpose/processed/{video_id}_{start_frame}.npy', {'body':body_sample, 'face':face_sample})
    
    
    return np.array(body_sample).astype(float), np.array(face_sample).astype(float)


def map_samples_to_labels(elan_label_path):
    elan_label = pd.read_csv(elan_label_path, index_col=0)

    vid_left_list = [] # left
    vid_right_list = [] # right
    
    all_samples = []
    
    
    for i in tqdm.tqdm(range(len(elan_label))):
        vid_main = elan_label['Name'].iloc[i]
        start_frame = elan_label['Start Frame'].iloc[i]

        # this determines the left and right people!
        if vid_main[-1] == '1':
            vid_left_list.append(vid_main[0:-1]+str(3))
            vid_right_list.append(vid_main[0:-1] + str(2))
        elif vid_main[-1] == '2':
            vid_left_list.append(vid_main[0:-1] + str(1))
            vid_right_list.append(vid_main[0:-1] + str(3))
        elif vid_main[-1] == '3':
            vid_left_list.append(vid_main[0:-1] + str(2))
            vid_right_list.append(vid_main[0:-1] + str(1))
        else: # if it's 0 (the scene camera)
            continue
            
        # get openpose data (for all people)
        # expensive so using futures
        with concurrent.futures.ThreadPoolExecutor(max_workers=18) as executor:
            futures = []
                
            futures.append(
                executor.submit(
                    get_openpose_from_json, vid_main, start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_openpose_from_json, vid_left_list[-1], start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_openpose_from_json, vid_right_list[-1], start_frame, 6
                )
            )
            
            
            futures.append(
                executor.submit(
                    get_gaze_from_csv, vid_main, start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_gaze_from_csv, vid_left_list[-1], start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_gaze_from_csv, vid_right_list[-1], start_frame, 6
                )
            )   
            
            futures.append(
                executor.submit(
                    get_time_from_csv, vid_main, start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_time_from_csv, vid_left_list[-1], start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_time_from_csv, vid_right_list[-1], start_frame, 6
                )
            )   
            
            futures.append(
                executor.submit(
                    get_count_from_csv, vid_main, start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_count_from_csv, vid_left_list[-1], start_frame, 6
                )
            )
            futures.append(
                executor.submit(
                    get_count_from_csv, vid_right_list[-1], start_frame, 6
                )
            )  
            
            
                
            concurrent.futures.wait(futures)
                
            body_main, face_main = futures[0].result()
            body_left, face_left = futures[1].result()
            body_right, face_right = futures[2].result()

            gaze_main, headpose_main = futures[3].result()
            gaze_left, headpose_left = futures[4].result()
            gaze_right, headpose_right = futures[5].result()
            
            time_since_last_lifted, time_since_last_bite, time_since_last_bite_zeroed = futures[6].result()
            time_since_last_lifted_left, time_since_last_bite_left, time_since_last_bite_zeroed_left = futures[7].result()
            time_since_last_lifted_right, time_since_last_bite_right, time_since_last_bite_zeroed_right = futures[8].result()
            
            num_lifted, num_bites = futures[9].result()
            num_lifted_left, num_bites_left = futures[10].result()
            num_lifted_right, num_bites_right = futures[11].result()
                
            # body_main, face_main = get_openpose_from_json(vid_main, start_frame, 6)
            # body_left, face_left = get_openpose_from_json(vid_left_list[-1], start_frame, 6)
            # body_right, face_right = get_openpose_from_json(vid_right_list[-1], start_frame, 6)
        # body_main = face_main = body_left = face_left = body_right = face_right = np.random.random((180, 28))
        
        # get speaking data (for all people)
        one, two, three = get_speaking_from_csv(vid_main[:2], start_frame, 6)
        if vid_main[-1] == '1':
            main_speaking = one
            left_speaking = three
            right_speaking = two
        elif vid_main[-1] == '2':
            main_speaking = two
            left_speaking = one
            right_speaking = three
        elif vid_main[-1] == '3':
            main_speaking = three
            left_speaking = two
            right_speaking = one
        
        # get gaze/headpose data (for all people)
        # gaze_main, headpose_main = get_gaze_from_csv(vid_main, start_frame, 6)
        # gaze_left, headpose_left = get_gaze_from_csv(vid_left_list[-1], start_frame, 6)
        # gaze_right, headpose_right = get_gaze_from_csv(vid_right_list[-1], start_frame, 6)

        # get time feats (only for the main person). we'll save all sides though
        # time_since_last_lifted, time_since_last_bite, time_since_last_bite_zeroed = get_time_from_csv(vid_main, start_frame, 6)
        # time_since_last_lifted_left, time_since_last_bite_left, time_since_last_bite_zeroed_left = get_time_from_csv(vid_left_list[-1], start_frame, 6)
        # time_since_last_lifted_right, time_since_last_bite_right, time_since_last_bite_zeroed_right = get_time_from_csv(vid_right_list[-1], start_frame, 6)

        # get count feats (only for the main person). we'll save all of them though
        # num_lifted, num_bites = get_count_from_csv(vid_main, start_frame, 6)
        # num_lifted_left, num_bites_left = get_count_from_csv(vid_left_list[-1], start_frame, 6)
        # num_lifted_right, num_bites_right = get_count_from_csv(vid_right_list[-1], start_frame, 6)
        
        person_main = {'body':body_main, 'face':face_main, 'speaking':main_speaking, 'gaze':gaze_main, 'headpose':headpose_main, 
                           'time_since_last_lifted':time_since_last_lifted, 'time_since_last_bite':time_since_last_bite, 'time_since_last_bite_zeroed':time_since_last_bite_zeroed, 
                           'num_lifted':num_lifted, 'num_bites':num_bites}
        
        person_left = {'body':body_left, 'face':face_left, 'speaking':left_speaking, 'gaze':gaze_left, 'headpose':headpose_left, 
                           'time_since_last_lifted':time_since_last_lifted_left, 'time_since_last_bite':time_since_last_bite_left, 'time_since_last_bite_zeroed':time_since_last_bite_zeroed_left, 
                           'num_lifted':num_lifted_left, 'num_bites':num_bites_left}
        person_right = {'body':body_right, 'face':face_right, 'speaking':right_speaking, 'gaze':gaze_right, 'headpose':headpose_right, 
                           'time_since_last_lifted':time_since_last_lifted_right, 'time_since_last_bite':time_since_last_bite_right, 'time_since_last_bite_zeroed':time_since_last_bite_zeroed_right, 
                           'num_lifted':num_lifted_right, 'num_bites':num_bites_right}
        
        sample = {'main':person_main, 'left':person_left, 'right':person_right, 'label':elan_label['label'].iloc[i], 'main_id':vid_main, 'start_frame':start_frame}
        
        all_samples.append(sample)

    np.save('all_samples.npy', all_samples)
    
def organize_data(person_features, global_features):
    samples = np.load('all_samples.npy', allow_pickle=True)
    person_main = []
    person_left = []
    person_right = []
    global_feats = []
    issue_count = 0
    
    labels = []
    
    for i in tqdm.tqdm(range(len(samples))):
        
        if samples[i]['start_frame'] < 0: # this should happen 16 times
            issue_count += 1
            continue
     
        if ('speaking' in person_features) and ('09' in samples[i]['main_id']):
            continue
        person_main.append([])
        person_left.append([])
        person_right.append([])
        global_feats.append([])
        labels.append(samples[i]['label'])

        for feature in person_features:          
            main_feature = samples[i]['main'][feature]
            left_feature = samples[i]['left'][feature]
            right_feature = samples[i]['right'][feature]
            
            if len(left_feature) != 180:
                # this only happens once in 14_2, so we ensure the rest of the values are the same...
                # just grow the size of it to 180
                print(feature, samples[i]['main_id'], left_feature.shape)
                diff = 180 - len(left_feature)
                to_add = left_feature[-1].reshape(1, -1).repeat(diff, axis=0)
                left_feature = np.concatenate([left_feature, to_add])
            
            person_main[-1].append(main_feature)
            person_left[-1].append(left_feature)
            person_right[-1].append(right_feature)

        person_main[-1] = np.concatenate(person_main[-1], axis=1)
        person_left[-1] = np.concatenate(person_left[-1], axis=1)
        person_right[-1] = np.concatenate(person_right[-1], axis=1)

                
        for feature in global_features:
            global_feats[-1].append(samples[i]['main'][feature])
    print(issue_count)
    return np.array(person_main), np.array(person_left), np.array(person_right), np.array(global_feats), np.array(labels)
            
import time
start = time.time()
# main, left, right, global_feats = organize_data(['body', 'face', 'gaze', 'headpose', 'speaking'], ['num_bites', 'time_since_last_bite'])
# main, left, right, global_feats, labels = organize_data(['face', 'gaze', 'headpose', 'speaking'], ['num_bites', 'time_since_last_bite'])


map_samples_to_labels('/home/aa2375/social-dining/src/preprocessing/labels.csv')




end = time.time()
total = end - start
print(total)
print((total/len(main)) * 9316)

def concatenate_op():
    p_path = "data/positive_op/"
    n_path = "data/negative_op/"
    main_list = []
    list_1 = []
    list_2 = []
    ops_main_positive = sorted(glob.glob(f'{p_path}/*_main.npy'))
    ops_1_positive = sorted(glob.glob(f'{p_path}/*_1.npy'))
    ops_2_positive = sorted(glob.glob(f'{p_path}/*_2.npy'))
    ops_main_negative = sorted(glob.glob(f'{n_path}/*_main.npy'))
    ops_1_negative = sorted(glob.glob(f'{n_path}/*_1.npy'))
    ops_2_negative = sorted(glob.glob(f'{n_path}/*_2.npy'))

    for i in range(len(ops_main_positive)):
        main_list.append(np.load(ops_main_positive[i], allow_pickle=True))
        list_1.append(np.load(ops_1_positive[i], allow_pickle=True))
        list_2.append(np.load(ops_2_positive[i], allow_pickle=True))

    for i in range(len(ops_main_negative)):
        main_list.append(np.load(ops_main_negative[i], allow_pickle=True))
        list_1.append(np.load(ops_1_negative[i], allow_pickle=True))
        list_2.append(np.load(ops_2_negative[i], allow_pickle=True))

    np.save("data/main_np.npy", np.array(main_list))
    np.save("data/dinner1_np.npy", np.array(list_1))
    np.save("data/dinner2_np.npy", np.array(list_2))

  4%|█████████████▊                                                                                                                                                                                                                                                                                                                                    | 379/9316 [00:38<12:01, 12.38it/s]

UH OH


 10%|█████████████████████████████████▏                                                                                                                                                                                                                                                                                                                | 916/9316 [01:31<11:15, 12.44it/s]

UH OH


 13%|██████████████████████████████████████████▌                                                                                                                                                                                                                                                                                                      | 1176/9316 [01:58<17:23,  7.80it/s]

...

 72%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                                             | 6744/9316 [11:46<03:43, 11.48it/s]

UH OH


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9316/9316 [16:04<00:00,  9.65it/s]


1168.3081545829773


NameError: name 'main' is not defined

In [4]:
from threading import Lock

pbar_mutex = Lock()

def get_openpose_from_json_pbar(v, s, t, pbar):
    get_openpose_from_json(v,s,t)
    with pbar_mutex:
        pbar.update(1)

def process_all_op(elan_label_path):
    elan_label = pd.read_csv(elan_label_path, index_col=0)

    jobs = []
    for i in tqdm.tqdm(range(len(elan_label))):
        vid_main = elan_label['Name'].iloc[i]
        start_frame = elan_label['Start Frame'].iloc[i]

        # this determines the left and right people!
        if vid_main[-1] == '1':
            left = vid_main[0:-1]+str(3)
            right = vid_main[0:-1] + str(2)
        elif vid_main[-1] == '2':
            left = vid_main[0:-1] + str(1)
            right = vid_main[0:-1] + str(3)
        elif vid_main[-1] == '3':
            left = vid_main[0:-1] + str(2)
            right = vid_main[0:-1] + str(1)
        else: # if it's 0 (the scene camera)
            continue
            
        jobs.append({'vid_main':vid_main, 'left':left, 'right':right, 'start_frame':start_frame})
    print("Starting")
    with tqdm.tqdm(total=len(jobs)*3) as pbar:      
        with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
            futures = []
            for job in jobs:
                futures.append(
                    executor.submit(
                        get_openpose_from_json_pbar, job['vid_main'], job['start_frame'], 6, pbar
                    )
                )
                futures.append(
                    executor.submit(
                        get_openpose_from_json_pbar, job['left'], job['start_frame'], 6, pbar
                    )
                )
                futures.append(
                    executor.submit(
                        get_openpose_from_json_pbar, job['right'], job['start_frame'], 6, pbar
                    )
                )
            # concurrent.futures.wait(futures) # I don't care about the output so we just end
            # for future in concurrent.futures.as_completed(futures):
            #     result = future.result()
    
process_all_op('/home/aa2375/social-dining/src/preprocessing/labels.csv')
    

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9316/9316 [00:00<00:00, 53892.39it/s]


Starting


 28%|████████████████████████████████████████████████████████████████████████████████████████████▏                                                                                                                                                                                                                                                 | 7718/27948 [00:06<00:15, 1277.58it/s]


KeyboardInterrupt: 

In [ ]:
main, left, right, global_feats, labels = organize_data(['face', 'gaze', 'headpose', 'speaking'], ['num_bites', 'time_since_last_bite'])
left.shape

In [5]:
# gaze/headpose features

def interpolate_gaze_headpose(df):
    l = []
    for i in range(1, len(df) - 1):
        pre = df['name'].iloc[i-1]
        cur = df['name'].iloc[i]
        if int(pre) != int(cur) - 1:
            diff = int(cur) - int(pre)
            for j in range(diff):
                should = int(pre) + j
                l.append(int(should))
        else:
            l.append(df.iloc[i-1]['name'])
    new_col = pd.DataFrame({"new_name": l})
    df = df.set_index("name")
    new_col = new_col.set_index("new_name")
    df_new = new_col.join(df)
    df_new = df_new.interpolate()
    return df_new

def get_gaze_from_csv(video_id, start_frame, window_size):
    """
    :param video_id: the id of video you want to get openpose
    :param start_frame: the frame when the bite action starts
    :param window_size: the window size of each sample in seconds
    """
    if os.path.exists(f"/home/aa2375/social-dining/data/rt-gene/{video_id}_interp.csv"):
        gaze_df = pd.read_csv(f"/home/aa2375/social-dining/data/rt-gene/{video_id}_interp.csv")
    else:
        gaze_df = pd.read_csv(f"/home/aa2375/social-dining/data/rt-gene/{video_id}.csv")

        gaze = np.array(gaze_df['gaze'])
        head_pose = np.array(gaze_df['headpose'])
        gazes = []
        head_poses = []
        for i in range(len(gaze)):
            gazes.append(np.array(ast.literal_eval(gaze[i])))
            head_poses.append(np.array(ast.literal_eval(head_pose[i])))
        gazes = np.array(gazes)
        head_poses = np.array(head_poses)

        gaze_df['gaze_phi'] = gazes[:,0]
        gaze_df['gaze_theta'] = gazes[:,1]
        gaze_df['headpose_phi'] = head_poses[:,0]
        gaze_df['headpose_theta'] = head_poses[:,1]
        # delete the gaze and headpose ones
        gaze_df = gaze_df.drop(columns=['gaze', 'headpose'])
        gaze_df = interpolate_gaze_headpose(gaze_df)
        gaze_df.to_csv(f"/home/aa2375/social-dining/data/rt-gene/{video_id}_interp.csv")

    
    start_id = start_frame
    end_id = start_frame + 30 * window_size
    
    data = gaze_df.loc[start_id : end_id-1]
    
    gaze_phi = np.array(data['gaze_phi'])
    gaze_theta = np.array(data['gaze_theta'])
    headpose_phi = np.array(data['headpose_phi'])
    headpose_theta = np.array(data['headpose_theta'])
    gaze = np.stack([gaze_phi, gaze_theta],axis=1)
    headpose = np.stack([headpose_phi, headpose_theta],axis=1)

    return gaze, headpose

gaze, headpose = get_gaze_from_csv('14_1', 64357, 6)
print(gaze.shape)
print(headpose.shape)

# for i in range(1, 30+1):
#     for j in range(1,3+1):
#         name = str(i).zfill(2) + '_' +str(j)
#         gaze, headpose = get_gaze_from_csv(name, 70, 6)


(45, 2)
(45, 2)


In [6]:
# Speaking features
def get_speaking_from_csv(session_id, start_frame, window_size):
    if session_id == '09':
        return None, None, None
    person_speaking = np.load(f"/home/aa2375/social-dining/data/person-speaking/{session_id}.npy")
    one = np.array(person_speaking == 1)
    two = np.array(person_speaking == 2)
    three = np.array(person_speaking == 3)
    
    start_id = start_frame
    end_id = start_frame + 30 * window_size
    
    one = one[start_id:end_id].reshape(-1,1)
    two = two[start_id:end_id].reshape(-1,1)
    three = three[start_id:end_id].reshape(-1,1)
    if len(one) == 0 or len(two) == 0 or len(three) == 0:
        print("UH OH")
    return one, two, three
    

one, two, three = get_speaking_from_csv('03', -5, 6)
print(one.shape, two.shape, three.shape)
print(len(one))

UH OH
(0, 1) (0, 1) (0, 1)
0


In [7]:
(np.array([1,0,1,1]) == 2).reshape(-1,1)

array([[False],
       [False],
       [False],
       [False]])

In [8]:
# time features
def get_time_from_csv(video_id, start_frame, window_size):
    """
    :param video_id: the id of video you want to get openpose
    :param start_frame: the frame when the bite action starts
    :param window_size: the window size of each sample in seconds
    """
    gaze_df = pd.read_csv(f"/home/aa2375/social-dining/data/time-features/{video_id}.csv")

    
    start_id = start_frame
    end_id = start_frame + 30 * window_size
    
    data = gaze_df.loc[start_id : end_id-1]
    
    time_since_last_lifted = np.array(data['Time since last food_lifted  (s)'])
    time_since_last_bite = np.array(data['Time since last food_to_mouth  (s)'])
    time_since_last_bite_zeroed = np.array(data['Time since last food_to_mouth (zeroed within) (s)'])

    return time_since_last_lifted, time_since_last_bite, time_since_last_bite_zeroed
time_since_last_lifted, time_since_last_bite, time_since_last_bite_zeroed = get_time_from_csv('01_2', 70, 6)

In [9]:
# count features
def get_count_from_csv(video_id, start_frame, window_size):
    """
    :param video_id: the id of video you want to get openpose
    :param start_frame: the frame when the bite action starts
    :param window_size: the window size of each sample in seconds
    """
    gaze_df = pd.read_csv(f"/home/aa2375/social-dining/data/count-features/{video_id}.csv")

    
    start_id = start_frame
    end_id = start_frame + 30 * window_size
    
    data = gaze_df.loc[start_id : end_id-1]
    
    num_lifted = np.array(data['Number of food_lifted annotations'])
    num_bites = np.array(data['Number of food_to_mouth annotations'])

    return num_lifted, num_bites
num_lifted, num_bites = get_count_from_csv('01_2', 70, 6)

In [11]:
# load positive csv

# add openposes to it
# mapping_op_sample_to_elan_label('data/positive_labels.csv', 'positive_samples')

# add gazes to it

# add speaking to it

# add count to it


# load negative csv

# add openposes to it
# mapping_op_sample_to_elan_label('data/negative_labels.csv', 'negative_samples')
